# 索引优化

## 上下文扩展

小块文本进行检索可以获得更高的精确度，但小块文本缺乏足够的上下文，可能导致大语言模型（LLM）无法生成高质量的答案。

而使用大块文本虽然上下文丰富，却容易引入噪音，降低检索的相关性。

为了解决这一矛盾，LlamaIndex 提出了**句子窗口检索（Sentence Window Retrieval）**。

在检索时聚焦于高度精确的单个句子，在送入LLM生成答案前，又智能地将上下文扩展回一个更宽的“窗口”，从而同时保证检索的准确性和生成的质量。

### 主要思路

为检索精确性而索引小块，为上下文丰富性而检索大块。

工作流程：

- 索引阶段：

  - 在构建索引时，文档被分割成单个句子。每个句子都作为一个独立的“节点（Node）”存入向量数据库。

  - 每个句子节点都会在元数据（metadata）中存储其上下文窗口，即该句子原文中的前N个和后N个句子。这个窗口内的文本不会被索引，仅仅是作为元数据存储。

- 检索阶段：

  - 当用户发起查询时，系统会在所有单一句子节点上执行相似度搜索。

- 后处理阶段：

  - 检索到最相关的句子节点后，系统使用 MetadataReplacementPostProcessor 的后处理模块读取到检索到的句子节点的元数据。

  - 用元数据中存储的完整上下文窗口来替换节点中原来的单一句子内容。

- 生成阶段：

  - 被替换了内容的、包含丰富上下文的节点被传递给LLM，用于生成最终的答案。


## 结构化索引

当一个查询可能只与其中一两个文档相关时，在整个文档库中进行无差别的向量搜索，不仅效率低下，还容易被不相关的文本块干扰，导致检索结果不精确。

结构化索引可以解决这个问题，其原理是在索引文本块的同时，为其附加结构化的元数据（Metadata）。

这些元数据可以是任何有助于筛选和定位信息的标签，例如：

- 文件名
  
- 文档创建日期
  
- 章节标题
  
- 作者
  
- 任何自定义的分类标签

在文本分块中，我们学习了基于文档结构的分块方法，就是实现结构化索引的一种前置步骤。

分块器会自动将Markdown文档的各级标题（如 Header 1, Header 2 等）提取并存入每个文本块的元数据中。

这些标题信息就是非常有价值的结构化数据，可以直接用于后续的元数据过滤。

例如，当用户查询“请总结一下2023年第二季度财报中关于AI的论述”时，系统可以：

1. 元数据预过滤：通过元数据筛选，只在 document_type == '财报'、year == 2023 且 quarter == 'Q2' 的文档子集中进行搜索。

2. 向量搜索：在经过滤的、范围更小的文本块集合中，执行针对查询“关于AI的论述”的向量相似度搜索。

### 另一种实现方式

将路由和检索彻底分离。

1. 创建两个独立的向量索引：

    - 摘要索引（用于路由）：为每个Excel工作表（例如，“1994年电影数据”）创建一个非常简短的摘要性Document，例如：“此文档包含1994年的电影信息”。然后，用所有这些摘要文档构建一个轻量级的向量索引。这个索引的唯一目的就是充当“路由器”。

    - 内容索引（用于问答）：将每个工作表的实际数据（例如，整个表格）转换为一个大的文本Document，并为其附加一个关键的元数据标签，如 {"sheet_name": "年份_1994"}。然后，用所有这些包含真实内容的文档构建一个向量索引。

2. 执行两步查询：

    - 第一步：路由。当用户提问（例如，“1994年评分人数最少的电影是哪一部？”）时，首先在“摘要索引”中进行检索。由于问题中的“1994年”与“此文档包含1994年的电影信息”这个摘要高度相关，检索器会快速返回其对应的元数据，告诉系统目标是 年份_1994 这个工作表。

    - 第二步：检索。拿到 年份_1994 这个目标后，系统会在“内容索引”中进行检索，但这次会附加一个元数据过滤器（MetadataFilter），强制要求只在 sheet_name == "年份_1994" 的文档中进行搜索。这样，LLM就能在正确的、经过筛选的数据范围内找到问题的答案。